# Rotation of Octahdera's 

This part of the code will be used in order to rotate the octahderas

In [1]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from sympy import Matrix
%matplotlib notebook


pi = np.pi

## Generalized Rotations around any given axis

In [2]:
np.set_printoptions(precision=10) ## this should satisfy the P2/m vs I2/m space group
## reducing the number of times we do approximation can also help with this. 
## we are copying from the VESTA software this could also be a reason error is introduced in the starting step only

def get_rotation_class(vec, angle):
    """
    Calculate the matrix classicaly for comparison
    
    Parameters:
        vec: list
            Three-dimensional array with the vector we are going to rotate around
        angle: float
            Angle we are going to rotate
    """
    a   = np.cos(angle / 2)
    b   = vec[0] * np.sin(angle / 2)
    c   = vec[1] * np.sin(angle / 2)    
    d   = vec[2] * np.sin(angle / 2)   
    
    mat = np.array([
        [a**2 + b**2 - c**2 - d**2, 2*(b*c - a*d), 2*(b*d + a*c)],
        [2*(b*c + a*d), a**2 + c**2 - b**2 - d**2, 2*(c*d - a*b)],
        [2*(b*d - a*c), 2*(c*d + a*b), a**2 + d**2 - b**2 - c**2],
    ])
    return mat

## Octahderal Rotation

## Generalized function

In [3]:
def Octahderal_tilt(Sn_gen, I_matrix, axis, angle):
    '''
    This will take in the position of an octahedra which is placed at some other location 
    '''
    axis    = axis / np.linalg.norm(axis)
    R = get_rotation_class(axis, angle)
    
    I_translate= []
    # translate I octahderal to origin
    for i in range(0,len(I_matrix)):
        I_translate.append(list(np.array(I_matrix[i]) - np.array(Sn_gen)))
        
#     print()
    I_rotated = []
    for i in range(0,len(I_matrix)):
        I_rotated.append(R @ I_translate[i])
        
    I_translate_back = []
    for i in range(0,len(I_matrix)):
        I_translate_back.append(list(np.array(I_rotated[i]) + np.array(Sn_gen)))
        
    return I_translate_back

In [4]:
def plot_octahdera(Sn,I, I_tilted):
    x=[]
    y=[]
    z=[]
    xt=[]
    yt=[]
    zt=[]    
    for i in range(0,len(I)):
        x.append(I[i][0])
    x.append(Sn[0])    
    for i in range(0,len(I)):
        y.append(I[i][1])
    y.append(Sn[1])
    for i in range(0,len(I)):
        z.append(I[i][2])
    z.append(Sn[2])

    for i in range(0,len(I)):
        xt.append(I_tilted[i][0])
    for i in range(0,len(I)):
        yt.append(I_tilted[i][1])
    for i in range(0,len(I)):
        zt.append(I_tilted[i][2])
    
    fig = plt.figure(figsize=(5,5))
    ax = plt.axes(projection='3d')
    x=np.array(x)
    y=np.array(y)
    z=np.array(z)

    xt=np.array(xt) 
    yt=np.array(yt)
    zt=np.array(zt)
    
    ax.plot3D(x, y, z,'*r')
    ax.plot3D(xt, yt, zt,'*b')

    plt.show()

In [5]:
## finding the exact angle 
def find_angle(a,b):
    angle = np.arccos(np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)))
    angle_deg = (angle/pi)*180
    return angle_deg # we will need angle in radians only in order to calcualte the exact difference that we wanna copensate

## way to use this

# # we will find angle between a and the octahderal arm
# a = [1,0,0]
# sn_i = list(np.array([0.71888 ,  0.77571,   0.50000])-np.array([0.50000,   0.50000,   0.50000  ]))


# find_angle(a,sn_i)

# resd_angle = find_angle(a,sn_i) - 45
# resd_angle

# Taking the inputs with higher precision

In [6]:
def final_coordinates_after_tilt(angle):
    
    # as we have to rotate by 11.5239
    angle_final = -pi/(180/(angle + 11.523920552984817))
    
    I = [[0.269495994,         0.265969992,         0.970390022],
         [0.730504036,         0.734030008,         0.029610001],
         [0.730504036,         0.265969992,         0.029610001],
         [0.269495994,         0.734030008,         0.970390022],
         [0.054763019,         0.000000000,         0.268579006],
         [0.945236981,         0.000000000,         0.731420994]]
        
    Sn1 = [0,0,0]
    I1 = [I[4]]

    Sn2 = [1,0,0]
    I2 = [I[2]]

    Sn3 = [1,1,0]
    I3= [I[1]]

    Sn4 = [0,0,1]
    I4 = [I[0]]

    Sn5 = [0,1,1]
    I5 = [I[3]]

    Sn6 = [1,0,1]
    I6 = [I[5]]
    
    I_t1 = Octahderal_tilt(Sn1, I1, [0,1,0], angle_final)
    I_t2 = Octahderal_tilt(Sn2, I2, [0,1,0], angle_final)
    I_t3 = Octahderal_tilt(Sn3, I3, [0,1,0], angle_final)
    I_t4 = Octahderal_tilt(Sn4, I4, [0,1,0], angle_final)
    I_t5 = Octahderal_tilt(Sn5, I5, [0,1,0], angle_final)
    I_t6 = Octahderal_tilt(Sn6, I6, [0,1,0], angle_final)

    I_t = [I_t1[0], I_t2[0], I_t3[0], I_t4[0], I_t5[0], I_t6[0]]
    
    Sn2 = [0.50000,   0.50000,   0.50000]

    I2 =  [[     0.769495964,         0.765969992,         0.470389992],
         [0.230504006,         0.234030008,         0.529609978],
         [0.230504006,         0.765969992,         0.529609978],
         [0.769495964,         0.234030008,         0.470389992],
         [0.554763019,         0.500000000,         0.768579006],
         [0.445236981,         0.500000000,         0.231420994]]

    I_t2 = Octahderal_tilt(Sn2, I2, [0,1,0], angle_final)
    # plot_octahdera(Sn2,I2,I_t2)

    
    return I_t,I_t2

In [34]:
# 10 deg tilt
I_t,I_t2 = final_coordinates_after_tilt(10)

In [37]:
I_t

[[-0.047594762886432086, 0.0, 0.26994149228995296],
 [0.7384338759017703, 0.265969992, -0.07133014436925551],
 [0.7384338759017703, 0.734030008, -0.07133014436925551],
 [0.2615661435677023, 0.265969992, 1.0713301767720276],
 [0.2615661435677023, 0.734030008, 1.0713301767720276],
 [1.047594762886432, 0.0, 0.7300585077100471]]

In [38]:
I_t2

[[0.7615661266664571, 0.765969992, 0.5713301378574042],
 [0.2384338564322977, 0.234030008, 0.4286698232279724],
 [0.2384338564322977, 0.765969992, 0.4286698232279724],
 [0.7615661266664571, 0.234030008, 0.5713301378574042],
 [0.45240523711356795, 0.5, 0.7699414922899529],
 [0.547594762886432, 0.5, 0.23005850771004704]]

In [42]:
# 40 deg tily
I_t,I_t2 = final_coordinates_after_tilt(40)

In [43]:
I_t

[[-0.17618901989172345, 0.0, 0.2099788084153644],
 [0.8091421639461278, 0.265969992, -0.19255677912850164],
 [0.8091421639461278, 0.734030008, -0.19255677912850164],
 [0.190857836713544, 0.265969992, 1.1925568169248617],
 [0.190857836713544, 0.734030008, 1.1925568169248617],
 [1.1761890198917233, 0.0, 0.7900211915846356]]

In [44]:
I_t2

[[0.690857841533948, 0.765969992, 0.6925567747731867],
 [0.309142163286456, 0.234030008, 0.3074431830751383],
 [0.309142163286456, 0.765969992, 0.3074431830751383],
 [0.690857841533948, 0.234030008, 0.6925567747731867],
 [0.32381098010827664, 0.5, 0.7099788084153644],
 [0.6761890198917234, 0.5, 0.2900211915846356]]

In [45]:
# 50 deg tily
I_t,I_t2 = final_coordinates_after_tilt(50)

In [46]:
I_t

[[-0.20997475021499293, 0.0, 0.17619385626659556],
 [0.8454788571240918, 0.265969992, -0.2227735244050164],
 [0.8454788571240918, 0.734030008, -0.2227735244050164],
 [0.15452113696228917, 0.265969992, 1.2227735617417157],
 [0.15452113696228917, 0.734030008, 1.2227735617417157],
 [1.209974750214993, 0.0, 0.8238061437334044]]

In [47]:
I_t2

[[0.654521149029022, 0.765969992, 0.7227735210674737],
 [0.3454788630377108, 0.234030008, 0.27722643825828436],
 [0.3454788630377108, 0.765969992, 0.27722643825828436],
 [0.654521149029022, 0.234030008, 0.7227735210674737],
 [0.29002524978500716, 0.5, 0.6761938562665956],
 [0.7099747502149929, 0.5, 0.32380614373340444]]

In [48]:
# 70 deg tily
I_t,I_t2 = final_coordinates_after_tilt(70)

In [49]:
I_t

[[-0.2575735713017965, 0.0, 0.09375247239822951],
 [0.930990655030272, 0.265969992, -0.26218798042312763],
 [0.930990655030272, 0.734030008, -0.26218798042312763],
 [0.06900932664284054, 0.265969992, 1.2621880134855716],
 [0.06900932664284054, 0.734030008, 1.2621880134855716],
 [1.2575735713017964, 0.0, 0.9062475276017705]]

In [50]:
I_t2

[[0.5690093518932704, 0.765969992, 0.7621879793913522],
 [0.43099067335715946, 0.234030008, 0.23781198651442842],
 [0.43099067335715946, 0.765969992, 0.23781198651442842],
 [0.5690093518932704, 0.234030008, 0.7621879793913522],
 [0.24242642869820358, 0.5, 0.5937524723982295],
 [0.7575735713017965, 0.5, 0.40624752760177046]]

In [51]:
# 80 deg tily
I_t,I_t2 = final_coordinates_after_tilt(80)

In [52]:
I_t

[[-0.2699403959727737, 0.0, 0.04760098041011931],
 [0.9775675270501668, 0.26596999199999993, -0.270188102863291],
 [0.9775675270501668, 0.734030008, -0.270188102863291],
 [0.022432449160139187, 0.26596999199999993, 1.2701881322410116],
 [0.022432449160139187, 0.734030008, 1.2701881322410116],
 [1.2699403959727738, 0.0, 0.9523990195898807]]

In [53]:
I_t2

[[0.5224324799473574, 0.765969992, 0.7701881030494511],
 [0.4775675508398608, 0.23403000800000007, 0.22981186775898854],
 [0.4775675508398608, 0.765969992, 0.22981186775898854],
 [0.5224324799473574, 0.23403000800000007, 0.7701881030494511],
 [0.23005960402722636, 0.5, 0.5476009804101193],
 [0.7699403959727738, 0.5, 0.45239901958988066]]

In [54]:
# 90 deg tily
I_t,I_t2 = final_coordinates_after_tilt(90)

In [55]:
I_t

[[-0.2741052183085494, 0.0, 3.1567195060214215e-06],
 [1.0248259984092631, 0.265969992, -0.26997869651973017],
 [1.0248259984092631, 0.734030008, -0.26997869651973017],
 [-0.024826026938925935, 0.265969992, 1.2699787213201001],
 [-0.024826026938925935, 0.734030008, 1.2699787213201001],
 [1.2741052183085493, 0.0, 0.999996843280494]]

In [56]:
I_t2

[[0.4751740084496265, 0.765969992, 0.7699786979181693],
 [0.5248260269389259, 0.234030008, 0.23002127867989997],
 [0.5248260269389259, 0.765969992, 0.23002127867989997],
 [0.4751740084496265, 0.234030008, 0.7699786979181693],
 [0.22589478169145066, 0.5, 0.500003156719506],
 [0.7741052183085494, 0.5, 0.499996843280494]]

In [23]:
# -10 deg tily
I_t,I_t2 = final_coordinates_after_tilt(-10)

In [24]:
I_t

[[0.047600980410119335, 0.0, 0.26994039597277364],
 [0.729811897136709, 0.265969992, 0.022432472949833207],
 [0.729811897136709, 0.734030008, 0.022432472949833207],
 [0.27018813224101146, 0.265969992, 0.9775675508398608],
 [0.27018813224101146, 0.734030008, 0.9775675508398608],
 [0.9523990195898806, 0.0, 0.7300596040272265]]

In [25]:
I_t2

[[0.7701881030494511, 0.765969992, 0.4775675200526426],
 [0.22981186775898854, 0.234030008, 0.5224324491601392],
 [0.22981186775898854, 0.765969992, 0.5224324491601392],
 [0.7701881030494511, 0.234030008, 0.4775675200526426],
 [0.5476009804101194, 0.5, 0.7699403959727735],
 [0.45239901958988066, 0.5, 0.23005960402722636]]

In [26]:
# -30 deg tily
I_t,I_t2 = final_coordinates_after_tilt(-30)

In [27]:
I_t

[[0.13705534295355953, 0.0, 0.2373805040053302],
 [0.7537785911286727, 0.265969992, 0.11348940296313105],
 [0.7537785911286727, 0.734030008, 0.11348940296313105],
 [0.24622144461390907, 0.265969992, 0.8865106093440966],
 [0.24622144461390907, 0.734030008, 0.8865106093440966],
 [0.8629446570464405, 0.0, 0.7626194959946699]]

In [28]:
I_t2

[[0.7462214066529663, 0.765969992, 0.3865105903976766],
 [0.25377855538609095, 0.234030008, 0.6134893906559034],
 [0.25377855538609095, 0.765969992, 0.6134893906559034],
 [0.7462214066529663, 0.234030008, 0.3865105903976766],
 [0.6370553429535595, 0.5, 0.7373805040053301],
 [0.36294465704644047, 0.5, 0.2626194959946698]]

In [32]:
# -40 deg tily
I_t,I_t2 = final_coordinates_after_tilt(-40)

In [33]:
I_t

[[0.17619385626659556, 0.0, 0.20997475021499293],
 [0.7772264755949836, 0.265969992, 0.15452114287590826],
 [0.7772264755949836, 0.734030008, 0.15452114287590826],
 [0.22277356174171564, 0.265969992, 0.8454788630377108],
 [0.22277356174171564, 0.734030008, 0.8454788630377108],
 [0.8238061437334044, 0.0, 0.7900252497850071]]

In [34]:
I_t2

[[0.7227735210674737, 0.765969992, 0.34547885097097797],
 [0.27722643825828436, 0.234030008, 0.6545211369622892],
 [0.27722643825828436, 0.765969992, 0.6545211369622892],
 [0.7227735210674737, 0.234030008, 0.34547885097097797],
 [0.6761938562665956, 0.5, 0.7099747502149929],
 [0.32380614373340444, 0.5, 0.2900252497850071]]

In [38]:
# -50 deg tily
I_t,I_t2 = final_coordinates_after_tilt(-50)

In [39]:
I_t

[[0.20997880841536437, 0.0, 0.17618901989172345],
 [0.8074432208714983, 0.265969992, 0.1908578360538721],
 [0.8074432208714983, 0.734030008, 0.1908578360538721],
 [0.19255681692486173, 0.265969992, 0.809142163286456],
 [0.19255681692486173, 0.734030008, 0.809142163286456],
 [0.7900211915846357, 0.0, 0.8238109801082766]]

In [40]:
I_t2

[[0.6925567747731867, 0.765969992, 0.309142158466052],
 [0.3074431830751383, 0.234030008, 0.690857836713544],
 [0.3074431830751383, 0.765969992, 0.690857836713544],
 [0.6925567747731867, 0.234030008, 0.309142158466052],
 [0.7099788084153643, 0.5, 0.6761890198917234],
 [0.29002119158463563, 0.5, 0.3238109801082766]]

In [44]:
# -60 deg tily
I_t,I_t2 = final_coordinates_after_tilt(-60)

In [45]:
I_t

[[0.23738366072483622, 0.0, 0.13704987535498986],
 [0.8435107064434009, 0.265969992, 0.22139541046206412],
 [0.8435107064434009, 0.734030008, 0.22139541046206412],
 [0.1564893306641967, 0.265969992, 0.7786045823250168],
 [0.1564893306641967, 0.734030008, 0.7786045823250168],
 [0.7626163392751638, 0.0, 0.8629501246450102]]

In [46]:
I_t2

[[0.656489288315846, 0.765969992, 0.27860458489740714],
 [0.3435106693358033, 0.234030008, 0.7213954176749832],
 [0.3435106693358033, 0.765969992, 0.7213954176749832],
 [0.656489288315846, 0.234030008, 0.27860458489740714],
 [0.7373836607248362, 0.5, 0.6370498753549898],
 [0.26261633927516376, 0.5, 0.36295012464501014]]

In [50]:
# -70 deg tily
I_t,I_t2 = final_coordinates_after_tilt(-70)

In [51]:
I_t

[[0.25757573062511224, 0.0, 0.09374653970617816],
 [0.8843330400126173, 0.265969992, 0.24520599735484958],
 [0.8843330400126173, 0.734030008, 0.24520599735484958],
 [0.11566699527872236, 0.265969992, 0.7547939890981451],
 [0.11566699527872236, 0.734030008, 0.7547939890981451],
 [0.7424242693748878, 0.0, 0.9062534602938219]]

In [52]:
I_t2

[[0.615666954020429, 0.765969992, 0.25479399898516897],
 [0.3843330047212776, 0.234030008, 0.7452060109018549],
 [0.3843330047212776, 0.765969992, 0.7452060109018549],
 [0.615666954020429, 0.234030008, 0.25479399898516897],
 [0.7575757306251122, 0.5, 0.5937465397061781],
 [0.24242426937488776, 0.5, 0.40625346029382187]]

In [54]:
# -80 deg tily
I_t,I_t2 = final_coordinates_after_tilt(-80)

In [55]:
I_t

[[0.2699414922899529, 0.0, 0.04759476288643213],
 [0.9286698556307444, 0.265969992, 0.2615661240982296],
 [0.9286698556307444, 0.734030008, 0.2615661240982296],
 [0.07133017677202763, 0.265969992, 0.7384338564322978],
 [0.07133017677202763, 0.734030008, 0.7384338564322978],
 [0.7300585077100472, 0.0, 0.952405237113568]]

In [56]:
I_t2

[[0.5713301378574042, 0.765969992, 0.23843387333354293],
 [0.42866982322797237, 0.234030008, 0.7615661435677022],
 [0.42866982322797237, 0.765969992, 0.7615661435677022],
 [0.5713301378574042, 0.234030008, 0.23843387333354293],
 [0.7699414922899528, 0.5, 0.547594762886432],
 [0.2300585077100471, 0.5, 0.4524052371135679]]

In [60]:
# -90 deg tily
I_t,I_t2 = final_coordinates_after_tilt(-90)

In [61]:
I_t

[[0.2741052183085494, 0.0, -3.1567195060214215e-06],
 [0.9751740015907367, 0.265969992, 0.26997869651973017],
 [0.9751740015907367, 0.734030008, 0.26997869651973017],
 [0.024826026938925935, 0.265969992, 0.7300212786799],
 [0.024826026938925935, 0.734030008, 0.7300212786799],
 [0.7258947816914507, 0.0, 1.000003156719506]]

In [62]:
I_t2

[[0.5248259915503735, 0.765969992, 0.23002130208183064],
 [0.47517397306107406, 0.234030008, 0.7699787213201],
 [0.47517397306107406, 0.765969992, 0.7699787213201],
 [0.5248259915503735, 0.234030008, 0.23002130208183064],
 [0.7741052183085493, 0.5, 0.49999684328049393],
 [0.2258947816914506, 0.5, 0.500003156719506]]